In [2]:
!nvidia-smi

Sun Jun 29 08:02:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8             10W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# !pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
# !pip uninstall -y bitsandbytes

In [6]:
# !pip install -U bitsandbytes

In [10]:
import bitsandbytes

In [11]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


2025-06-29 08:04:30.926037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751184271.095194      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751184271.144496      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


#### Finetuning Llama2 model

In [12]:
from datasets import load_dataset

ds = load_dataset("mlabonne/guanaco-llama2")

README.md:   0%|          | 0.00/816 [00:00<?, ?B/s]

data/train-00000-of-00001-0e1c2f1cce3a25(…):   0%|          | 0.00/8.99M [00:00<?, ?B/s]

data/test-00000-of-00001-f1915360acfe3af(…):   0%|          | 0.00/476k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9846 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/518 [00:00<?, ? examples/s]

In [13]:
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 9846
    })
    test: Dataset({
        features: ['text'],
        num_rows: 518
    })
})

In [14]:
bnb_4bit_compute_dtype = "float16"
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

In [15]:
use_4bit=True
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [16]:
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [19]:
# Load base model
model_name="NousResearch/Llama-2-7b-chat-hf"
# device_map = {"": 0}
# device_map="auto"

In [20]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

In [21]:
print(model.hf_device_map)


{'': 0}


In [23]:
def print_trainable_parameters(model):
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total = sum(p.numel() for p in model.parameters())
    print(f"Trainable params: {trainable:,}")
    print(f"Total params:     {total:,}")
    print(f"Trainable %:      {100 * trainable / total:.4f}%")

print_trainable_parameters(model)


Trainable params: 262,410,240
Total params:     3,500,412,928
Trainable %:      7.4966%


In [24]:
# # Load LLaMA tokenizer
# tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
# tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side = "right"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map=None  # Ensures tokenizer does not try to load on GPU
)

tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [25]:
lora_r = 16
lora_alpha = 16
lora_dropout = 0.1

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [27]:
# Set training parameters
learning_rate = 2e-4


training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="adamw_torch",
    save_strategy="no", 
    # save_steps=0,
    eval_strategy="epoch",
    logging_steps=25,
    learning_rate=learning_rate,
    weight_decay=0.001,
    fp16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type= "cosine",
    report_to="tensorboard"
)

In [29]:
def format_text(example):
    return example["text"]

trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    peft_config=peft_config,
    formatting_func=format_text
)

Applying formatting function to train dataset:   0%|          | 0/9846 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/9846 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/9846 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/9846 [00:00<?, ? examples/s]

Applying formatting function to eval dataset:   0%|          | 0/518 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/518 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/518 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/518 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [30]:
# import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [31]:
# Train model
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [32]:
# Save trained model
# trainer.model.save_pretrained('\results')
trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")

('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/tokenizer.model',
 './final_model/added_tokens.json',
 './final_model/tokenizer.json')

In [34]:
trainer.model.save_pretrained("./model")

In [36]:
prompt = "Explain the theory of relativity in simple terms."
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=100,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


Explain the theory of relativity in simple terms. [/INST] The Theory of Relativity, developed by Albert Einstein, is a fundamental concept that explains how space and time are intertwined as a single entity called spacetime. Here's an easy-to-understand explanation:
Imagine you are traveling at high speed on a train, such as from New York to Los Angeles. You look out the window and see trees passing by really fast or slow depending on where they are relative to your motion. Now imagine this


In [44]:
ds['train'][2]['text']

'<s>[INST] Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML? [/INST] Sure! Let\'s say you want to build a model which can distinguish between images of cats and dogs. You gather your dataset, consisting of many cat and dog pictures. Then you put them through a neural net of your choice, which produces some representation for each image, a sequence of numbers like [0.123, 0.045, 0.334, ...]. The problem is, if your model is unfamiliar with cat and dog images, these representations will be quite random. At one time a cat and a dog picture could have very similar representations (their numbers would be close to each other), while at others two cat images may be represented far apart. In simple terms, the model wouldn\'t be able to tell cats and dogs apart. This is where contrastive learning comes in.\n\nThe point of contrastive learning is to take pairs of samples (in this case images of cats and dogs), then train the model to "pu

In [46]:
prompt = "Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=1000,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)


Can you explain contrastive learning in machine learning in simple terms for someone new to the field of ML?
Contrastive Learning is a type of self-supervised learning where two neural networks are trained on similar and dissimilar data. The goal is to learn representations that can be used to complete tasks without training on task specific labels or textures, such as color classification or object recognition. Contrastive learning has been shown to achieve state-of-the-art results on some benchmark datasets with only few labeled examples per class.


In [ ]:
metrics = trainer.evaluate()
print(metrics)

In [ ]:
metrics = trainer.evaluate(eval_dataset=ds["test"])
print(metrics)

In [ ]:
import torch

# Clear cache
torch.cuda.empty_cache()

# Optionally delete specific variables
del model  # Replace with your tensor or model
torch.cuda.empty_cache()

# To make sure memory is released
import gc
gc.collect()
torch.cuda.empty_cache()


In [ ]:
import tensorflow as tf
from numba import cuda  # Only if using numba with CUDA

# Reset default graph (if needed)
tf.keras.backend.clear_session()

# If you used Numba or want a full reset
cuda.select_device(0)
cuda.close()
